<a href="https://colab.research.google.com/github/hertie-data-science-lab/tutorial-new-group-2-1/blob/xiaohan-modeling/tutorial_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Transfer Learning for Flood Mapping Using Sentinel-1 Radar Imagery


# GRAD-E1394 Deep Learning - Assignment 3

Authors:


*   Aditi Joshi
*   Elena Murray
*   Leticia Figueiredo Collado
*   Sattiki Ganguly
*   Xiaohan Wu







Test - check commit.

In [ ]:
!pip install segmentation-models-pytorch --quiet
!pip install pretrainedmodels --quiet
!pip install efficientnet-pytorch --quiet

In [ ]:
!gsutil ls gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/ > chip_list.txt

with open("chip_list.txt") as f:
    chip_dirs = [line.strip() for line in f]

country_dirs = [d for d in chip_dirs if "india" in d.lower()]

chip_ids = [d.rstrip("/").split("/")[-1].replace("_label", "")
            for d in country_dirs]

print(f"Found {len(chip_ids)} India chips")
print("Example:", chip_ids[:5])


In [ ]:
import torch
import numpy as np
import rasterio
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms.functional as F
import torchvision.transforms as T
import random

# GCS streaming prefixes
HTTP_PREFIX = "https://storage.googleapis.com/sen1floods11/v1.1"
S1_PREFIX    = f"/vsicurl/{HTTP_PREFIX}/data/flood_events/HandLabeled/S1Hand"
LABEL_PREFIX = f"/vsicurl/{HTTP_PREFIX}/data/flood_events/HandLabeled/LabelHand"

class Sentinel1FloodDataset(Dataset):
    def __init__(self, id_list):
        self.ids = id_list

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        cid = self.ids[idx]

        s1_path    = f"{S1_PREFIX}/{cid}_S1Hand.tif"
        label_path = f"{LABEL_PREFIX}/{cid}_LabelHand.tif"

        # --- Load Sentinel-1 SAR image (VV/VH) ---
        with rasterio.open(s1_path) as src:
            s1_img = src.read().astype("float32")  # (2, 512, 512)

        # Robust SAR normalization
        s1_img = np.nan_to_num(s1_img)
        s1_img = np.clip(s1_img, -50, 50)
        s1_img = np.log1p(s1_img - s1_img.min())
        s1_img = (s1_img - s1_img.mean()) / (s1_img.std() + 1e-6)

        # --- Load flood mask ---
        with rasterio.open(label_path) as src:
            mask_raw = src.read(1).astype("int16")

        valid_mask = (mask_raw != -1)
        label = (mask_raw == 1).astype("float32")

        x = torch.tensor(s1_img, dtype=torch.float32)
        y = torch.tensor(label, dtype=torch.float32)[None, ...]
        valid = torch.tensor(valid_mask, dtype=torch.bool)[None, ...]

        return x, y, valid

In [ ]:
def tensor_to_pil_pair(x, y, valid):
    vv = x[0].cpu().numpy()
    vh = x[1].cpu().numpy()
    label_arr = y[0].cpu().numpy()
    valid_arr = valid[0].cpu().numpy().astype(np.uint8)

    vv_pil = Image.fromarray((vv * 255).astype(np.uint8))
    vh_pil = Image.fromarray((vh * 255).astype(np.uint8))
    label_pil = Image.fromarray((label_arr * 255).astype(np.uint8))
    valid_pil = Image.fromarray((valid_arr * 255).astype(np.uint8))

    return vv_pil, vh_pil, label_pil, valid_pil

In [ ]:
def augment_train(x, y, valid):
    vv_pil, vh_pil, label_pil, valid_pil = tensor_to_pil_pair(x, y, valid)

    # Random flip (safe)
    if random.random() > 0.5:
        vv_pil    = F.hflip(vv_pil)
        vh_pil    = F.hflip(vh_pil)
        label_pil = F.hflip(label_pil)
        valid_pil = F.hflip(valid_pil)

    if random.random() > 0.5:
        vv_pil    = F.vflip(vv_pil)
        vh_pil    = F.vflip(vh_pil)
        label_pil = F.vflip(label_pil)
        valid_pil = F.vflip(valid_pil)

    # Convert back to tensors
    vv    = F.to_tensor(vv_pil).squeeze(0)
    vh    = F.to_tensor(vh_pil).squeeze(0)
    label = F.to_tensor(label_pil).round().squeeze(0)
    valid = F.to_tensor(valid_pil).round().squeeze(0).bool()

    x_aug = torch.stack([vv, vh], dim=0)
    y_aug = label.unsqueeze(0)
    valid_aug = valid.unsqueeze(0)

    return x_aug, y_aug, valid_aug

In [ ]:
def preprocess_test(x, y, valid):
    return x, y, valid

In [ ]:
class AugmentedSentinel1Dataset(torch.utils.data.Dataset):
    def __init__(self, base_dataset, augment=False):
        self.base = base_dataset
        self.augment = augment

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        x, y, valid = self.base[idx]

        if self.augment:
            return augment_train(x, y, valid)

        else:
            return preprocess_test(x, y, valid)

In [ ]:
valid_ids = sorted(chip_ids)

np.random.seed(42)
np.random.shuffle(valid_ids)

n = len(valid_ids)
train_ids = valid_ids[:int(0.7*n)]
val_ids   = valid_ids[int(0.7*n):int(0.85*n)]
test_ids  = valid_ids[int(0.85*n):]

print(f"Train: {len(train_ids)}  Val: {len(val_ids)}  Test: {len(test_ids)}")

In [ ]:
batch_size = 4

train_ds = AugmentedSentinel1Dataset(Sentinel1FloodDataset(train_ids), augment=True)
val_ds   = AugmentedSentinel1Dataset(Sentinel1FloodDataset(val_ids),   augment=False)
test_ds  = AugmentedSentinel1Dataset(Sentinel1FloodDataset(test_ids),  augment=False)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

In [ ]:
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

# U-Net with ResNet34 encoder pre-trained on ImageNet  ← TRANSFER LEARNING
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",   # this is the transfer part
    in_channels=2,                # VV + VH
    classes=1                     # binary mask
).to(device)

criterion = nn.BCEWithLogitsLoss()

In [ ]:
def compute_iou_from_logits(logits, target, valid):
    """
    logits: (B,1,H,W)
    target: (B,1,H,W) with 0/1
    valid:  (B,1,H,W) bool
    """
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()

    v = valid.bool()
    if v.sum() == 0:
        return torch.tensor(0.0, device=logits.device)

    p = preds[v]
    t = target[v]

    intersection = (p * t).sum()
    union = p.sum() + t.sum() - intersection
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou


def compute_accuracy_from_logits(logits, target, valid):
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()

    v = valid.bool()
    if v.sum() == 0:
        return torch.tensor(0.0, device=logits.device)

    p = preds[v]
    t = target[v]

    correct = (p == t).float().sum()
    acc = correct / p.numel()
    return acc

In [ ]:
def train_one_epoch(model, dl, optimizer):
    model.train()
    total_loss = 0.0
    total_iou = 0.0
    total_acc = 0.0
    n_batches = 0

    for x, y, valid in dl:
        x = x.to(device)
        y = y.to(device)
        valid = valid.to(device)

        optimizer.zero_grad()
        logits = model(x)  # (B,1,H,W)

        if valid.sum() == 0:
            continue

        loss = criterion(logits[valid], y[valid])
        loss.backward()
        optimizer.step()

        iou = compute_iou_from_logits(logits, y, valid).item()
        acc = compute_accuracy_from_logits(logits, y, valid).item()

        total_loss += loss.item()
        total_iou  += iou
        total_acc  += acc
        n_batches  += 1

    if n_batches == 0:
        return 0.0, 0.0, 0.0

    return (
        total_loss / n_batches,
        total_iou  / n_batches,
        total_acc  / n_batches,
    )


@torch.no_grad()
def validate_one_epoch(model, dl):
    model.eval()
    total_loss = 0.0
    total_iou = 0.0
    total_acc = 0.0
    n_batches = 0

    for x, y, valid in dl:
        x = x.to(device)
        y = y.to(device)
        valid = valid.to(device)

        logits = model(x)

        if valid.sum() == 0:
            continue

        loss = criterion(logits[valid], y[valid])
        iou = compute_iou_from_logits(logits, y, valid).item()
        acc = compute_accuracy_from_logits(logits, y, valid).item()

        total_loss += loss.item()
        total_iou  += iou
        total_acc  += acc
        n_batches  += 1

    if n_batches == 0:
        return 0.0, 0.0, 0.0

    return (
        total_loss / n_batches,
        total_iou  / n_batches,
        total_acc  / n_batches,
    )

In [ ]:
# Freeze encoder → only train decoder/head (classic transfer learning warmup)
for p in model.encoder.parameters():
    p.requires_grad = False

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3,
)

print("=== Stage 1: Train Decoder Only (Frozen Encoder) ===")
num_epochs_stage1 = 3

for epoch in range(num_epochs_stage1):
    tr_loss, tr_iou, tr_acc = train_one_epoch(model, train_dl, optimizer)
    va_loss, va_iou, va_acc = validate_one_epoch(model, val_dl)

    print(f"Epoch {epoch+1}/{num_epochs_stage1}")
    print(f"  Train - Loss: {tr_loss:.4f}, IoU: {tr_iou:.4f}, Acc: {tr_acc:.4f}")
    print(f"  Val   - Loss: {va_loss:.4f}, IoU: {va_iou:.4f}, Acc: {va_acc:.4f}")

Afrer training, turn the encoder into a reusable embedding

In [ ]:
def extract_embedding(model, x_batch):
    """
    Extracts a reusable embedding from the deepest encoder feature map.
    """
    model.eval()
    with torch.no_grad():
        # SMP encoders return a list of feature maps → take deepest one
        feat_list = model.encoder(x_batch)      # list of tensors
        feats = feat_list[-1]                   # (B, C, H', W')

        # Global average pooling over spatial dims
        pooled = feats.mean(dim=(2, 3))         # (B, C)
    return pooled

Example: does this chip contain any flooded pixels?


In [ ]:
def compute_embeddings(dataloader, model, device="cuda"):
    """
    Returns:
      Z: (N, C) numpy array of embeddings
      Y: (N,) numpy array of chip-level labels (0/1)
    """
    all_z = []
    all_y = []

    model.eval()
    with torch.no_grad():
        for x, y, valid in dataloader:
            x = x.to(device)
            y = y.to(device)

            # 1) Compute embeddings
            z = extract_embedding(model, x)  # (B, C)
            all_z.append(z.cpu().numpy())

            # 2) Create simple chip-level label:
            #    1 if any flood pixel exists, else 0
            #    (you can refine this, e.g. >1% flood coverage)
            y_flat = y.view(y.size(0), -1)
            chip_label = (y_flat.max(dim=1).values > 0.5).float()
            all_y.append(chip_label.cpu().numpy())

    Z = np.concatenate(all_z, axis=0)
    Y = np.concatenate(all_y, axis=0)
    return Z, Y

In [ ]:
Z_train, Y_train = compute_embeddings(train_dl, model, device=device)
Z_val,   Y_val   = compute_embeddings(val_dl,   model, device=device)
Z_test,  Y_test  = compute_embeddings(test_dl,  model, device=device)

print(Z_train.shape, Y_train.shape)

47 instead of 48, one batch was skipped during embedding extraction because it contained no valid pixels.

Downstream task: chip-level classification

We use `Z_train` which is an embedding matrix of shape (47. 512), with each row representing 512-dimensional feature vector produced by the encoder.

`Y_train` is the labels, each label = 0 (no flood) or 1 (flood exists somewhere in the chip)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

clf = LogisticRegression(max_iter=1000)
clf.fit(Z_train, Y_train)

y_pred = clf.predict(Z_test)
print("Chip-level flood presence accuracy:", accuracy_score(Y_test, y_pred))
print(classification_report(Y_test, y_pred))

A more complicated downstream task: few-shot flood classification (simulate non-experts with tiny labels)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit

K = 6  # must be >= 2
sss = StratifiedShuffleSplit(n_splits=1, train_size=K)

for few_idx, _ in sss.split(Z_train, Y_train):
    pass

Z_few = Z_train[few_idx]
Y_few = Y_train[few_idx]

print("Few-shot indices:", few_idx)
print("Few-shot labels:", Y_few)

clf = LogisticRegression(max_iter=1000)
clf.fit(Z_few, Y_few)

y_pred = clf.predict(Z_test)

print(f"\nFew-shot ({K}) accuracy:", accuracy_score(Y_test, y_pred))
print("\nClassification report:\n", classification_report(Y_test, y_pred))

We should increase the sample size because currently all labels are 1.